In [ ]:
import io, os
input=io.BytesIO(os.read(0,os.fstat(0).st_size)).readline

def ett(G, N, s) :
  disc = 0
  IN = [0] * (N+1)
  OUT = [0] * (N+1)
  DEP = [0] * (N+1)
  DEP[s] = 1
  S = [(s, -1)]
  while S:
    u, p = S[-1]

    if IN[u] == 0:
      disc += 1
      IN[u] = disc
      for v in G[u]:
        if v == p: continue
        DEP[v] = DEP[u] + 1
        S.append((v, u))
    else:
      S.pop()
      OUT[u] = disc
  
  return IN, OUT, DEP

class ST:
  def __init__(self, L, f, default=0):
    self._def = default
    self._f = f
    self._len = len(L)
    self._size = _size = 1 << (self._len - 1).bit_length()

    self.seg = [default] * (2 * _size)
    self.seg[_size:_size + self._len] = L
    for i in reversed(range(_size)):
      self.seg[i] = f(self.seg[i + i], self.seg[i + i + 1])

  def __getitem__(self, i):
    return self.seg[i + self._size]

  def __setitem__(self, i, v):
    i += self._size
    self.seg[i] = v
    i //= 2
    while i:
      self.seg[i] = self._f(self.seg[2 * i], self.seg[2 * i + 1])
      i //= 2

  def query(self, s, e):
    s += self._size
    e += self._size

    l = r = self._def
    while s < e: 
      if s & 1:
        l = self._f(l, self.seg[s])
        s += 1
      if e & 1:
        e -= 1
        r = self._f(self.seg[e], r)
      s //= 2
      e //= 2

    return self._f(l, r)

def sol() :
  N, C = map(int, input().split())
  G = [[] for _ in range(N+1)]
  for _ in range(N-1) :
    u, v = map(int, input().split())
    G[u].append(v)
    G[v].append(u)
  
  IN, OUT, DEP = ett(G, N, C)
  st = ST([0] * (N+1), lambda x, y: x + y)
  ans = []
  for _ in range(int(input())):
    q, x = map(int, input().split())
    if q == 1 :
      st[IN[x]] += 1
    else :
      ans.append(st.query(IN[x], OUT[x]+1) * DEP[x])

  print(*ans, sep="\n")
sol()

### 풀이
- ETT를 통해 선형화된 DAG 정점의 특성을 응용한 문제.
- ETT쿼리를 $E(x)$ 질의 쿼리를 $Q(x)$ 라고 하자. 
  - $E(x)$ 를 수행하면 모든 서브 노드에 대한 $\sum Q$ 를 구하게 되므로, 각 $Q(x)$ 에는 물이 채워진 횟수를 저장하도록 설계하면 될 것이다.
  - (핵심) 이때, 문제에 정의에 의해 $x = \text{트리의 깊이}$ 이다.
- 루트 $C$ 인 트리 $G$ 에 대해 각 노드의 깊이 $D[x]$ 를 $O(N)$으로 구한 뒤, $E(x) * D[x]$ 를 출력한다.

- 트리 높이도 구해두는 귀찮지 않은 예시를 하나 또 만들어뒀다.
- 도대체 며칠을 이거 하날 못풀어서 유기를 했는지..